## Code - To put in a python file at the end

In [5]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import re
import json
import numpy as np
import graphviz
from IPython.display import IFrame

if not os.path.isdir('./tree-sitter-go'):
    print("Cloning https://github.com/tree-sitter/tree-sitter-go")
    os.system("git clone https://github.com/tree-sitter/tree-sitter-go")
            
Language.build_library('build/my-languages.so', ['tree-sitter-go'])
GO_LANGUAGE = Language('build/my-languages.so', 'go')
parser = Parser()
parser.set_language(GO_LANGUAGE)

class ToggleCatcher():
    
    def __init__(self, system_name):
        
        self.system_name = system_name
        
        # configuration file containing the json - see the HowTo.md 
        # to add your own configuration
        self.config_file_path = "./config/"+self.system_name+'.json'
        
        # directory to export the results
        self.results_dir = "./results/"
        
        # inputs - properties of the current system
        attributes = json.load(open(self.config_file_path))
        ## directory in which the source code is located
        ## e.g. "./kops/" for Kops
        self.directory = attributes["directory"]
        ## if the directory of the system does not exist, we clone the repository
        if not os.path.isdir(self.directory):
            print("Cloning "+attributes["url"])
            os.system("git clone "+attributes["url"])

        ## if we have access to the keywords, we use them
        if "keywords" in attributes:
            self.keywords = [k.lower() for k in attributes["keywords"]]
        else:
            ## else we find them with the regular expression
            ## the file listing the Feature Toggles (FT) and their name 
            ## aka our keywords to search in the code
            ## e.g. "./kops/pkg/featureflag/featureflag.go" for Kops
            self.ft_file = attributes["ft_file"]
            ## the regular expression to use to find the names of FTs
            ## e.g. "[N|n]ew.*,*Bool*" for Kops
            self.reg_exp = attributes["reg_exp"]
            self.sep = None
            if "sep_reg_exp" in attributes:
                self.sep = attributes['sep_reg_exp']
            self.keywords = self.get_ft_keywords()
        ## the way ft are expressed in the code
        ## depends on the library used by the developers
        ## e.g. "featureflag." for Kops
        self.feature_structure = attributes["feature_structure"]
        
        # outputs - measures on the system
        ## counts the number of files with FT
        self.count_file = dict()
        ## counts the occurrences of FTs in the files
        self.occurences = dict()
        ## lists keywords per file
        self.kw_file = dict()
        ## the resulting list of interesting statements
        self.statements = []
        ## initialize the dicts
        for kw in sorted(self.keywords):
            self.count_file[kw] = 0
            self.occurences[kw] = 0
        
        # files with fts
        self.file_interests = self.list_kw_files()
        
        # launches an analyse
        self.analyse_all_files()

    def get_ft_keywords(self):
        ## output : search keywords in the file self.ft_file containing all the feature toggles
        ## uses the regular_expression self.reg_exp to search in the file
        
        with open(self.ft_file, 'r') as f:
            catch_feat = re.findall(self.reg_exp, f.read())
        
        if self.sep:
            keywords = [feature.split(self.sep)[1] for feature in catch_feat]
        else:
            keywords = [feature for feature in catch_feat]

        return [k.lower() for k in keywords]
    
    def list_kw_files(self):
        ### output : lists the files with feature toggles based on the self.keywords list of FTs
        
        go_folders = [x[0] for x in os.walk(self.directory)]

        go_files = []
        for dir_name in go_folders:
            files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
            # '.go' could be an input the language in the next version in the configuration file
            go_files.extend(files)

        go_file_interests = []

        for file in go_files:
            s = ""
            with open(file, "r") as f:
                s+=f.read().lower()
            kws = [k for k in self.keywords if k in s]
            if len(kws) > 0 and self.feature_structure in s:
                self.kw_file[file] = []
                for k in kws:
                    self.kw_file[file].append(k)
                    if self.feature_structure+k in s:
                        self.count_file[k]+=1
                    self.occurences[k]+=s.count(self.feature_structure+k)
                go_file_interests.append(file)

        return go_file_interests

    
    def get_code(self, node):
        ### input : a node of the ast
        ### output : get the "code" of the node of the ast, 
        ### i.e. the string content of the related part in the code
        
        code = self.source[node.start_byte:node.end_byte].decode('utf8').lower()
        code = code.replace('\n', '').replace('\t','')
        return code

    
    def get_id(self, node):
        ### input : a node of the ast
        ### output : get the "code" of the node, i.e. the related part of the code
        
        node_type = node.type
        if node_type not in self.type_nodes:
            self.type_nodes[node_type]=1
        else:
            self.type_nodes[node_type]+=1
        return node_type+str(self.type_nodes[node_type])

    
    def process_node(self, root_id, node):
        ### input : a parent node and a child node
        ### output : analysis of the parent and starts the analyses of the grandchildren
        ### works recursively
        ### analyse = isolates the conditions of the if statements containing at least one feature toggle
        ### @Aaron to replace with your code when you have the time
        
        node_id = self.get_id(node)
        node_content = self.get_code(node)
        for kw in self.keywords:
            if kw in node_content and node.type == 'if_statement':
                for c in node.children:
                    if c.type in ['binary_expression', 'unary_expression', 'call_expression']:
                        final_code = self.get_code(c)
                        if self.feature_structure in final_code:
                            self.statements.append(final_code)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                self.process_node(node_id, node.children[i])
    
    
    def analyse_file(self, filename):
        ### input : a filename
        ### output: process the ast of one file
        
        s = ""
        with open(filename, "r") as f:
            s+=f.read()+"\n"
        s = s.lower()

        self.source = bytes(s, "utf8")
        ast = parser.parse(self.source)

        root_node = ast.root_node

        self.type_nodes = dict()

        for i in range(len(root_node.children)):
            self.process_node("root", root_node.children[i])
    
    def analyse_all_files(self):
        ### output : analyse all the .go files of the project
        ### and put the results in self.statements
        
        for fi in self.file_interests:
            self.analyse_file(fi)
            
    def export_results(self):
        #### output: export results to the ./results dir
        print("------\nExporting", system, "results\n------")
        print("Saving statements to ", 
              self.results_dir+"statements/"+system+".txt")
        np.savetxt(self.results_dir+"statements/"+self.system_name+".txt", 
                   self.statements, fmt="%s", delimiter="\n")
        print("Saving other results to ", 
              self.results_dir+'kw_file/'+self.system_name+'.json',
              self.results_dir+'occurences/'+self.system_name+'.json',
              self.results_dir+'count_file/'+self.system_name+'.json')
        with open(self.results_dir+'kw_file/'+self.system_name+'.json', "w") as outfile:
            json.dump(tc.kw_file, outfile)
        with open(self.results_dir+'occurences/'+self.system_name+'.json', "w") as outfile:
            json.dump(tc.occurences, outfile)
        with open(self.results_dir+'count_file/'+self.system_name+'.json', "w") as outfile:
            json.dump(tc.count_file, outfile)
        print("\n")
        
    def process_statement(self, dico, logic_link, statement):
        ### inputs: a dico to increment, a statement to analyse and the type of logical links
        ### between the expressions
        ### output: the dico incremented
        tab = statement.split(logic_link)
        res = ['expr']*len(tab)
        for i in range(len(tab)):
            t = tab[i]
            for kw in self.keywords:
                if kw in t:
                    res[i] = kw
        for i in range(len(res)):
            first = res[i]
            for j in range(i+1, len(res)):
                second = res[j]
                index = (min(first, second), max(first, second))
                if index not in dico:
                    dico[index] = 1
                else:
                    dico[index]+=1
        return dico
        
    def analyse_statements(self):
        ### output: three distionaries, containing the relationships between feature toggles
        ### and_rel the relations of intersection
        ### or_rel the relations of union
        ### alone the number of solely call of FT
        
        and_rel = dict()
        or_rel = dict()
        alone = dict()

        for st in self.statements:
            if '&&' in st:
                and_rel = self.process_statement(and_rel, '&&', st)
            elif '||' in st:
                or_rel = self.process_statement(or_rel, '||', st)
            else:
                res = 'expr'
                for kw in self.keywords:
                    if kw in st:
                        res = kw
                if res not in alone:
                    alone[res] = 1
                else:
                    alone[res]+=1
        
        return (alone, or_rel, and_rel)
    
    def build_ftm(self):
        ### input : three dictionaries, see analyse_statements above
        ### output : graphical representation of FTM
        
        alone, or_rel, and_rel = self.analyse_statements()
        
        ftm = graphviz.Graph(comment='FTM '+system)

        for al in alone:
            if al != 'expr':
                ftm.node(al, al)
                ftm.edge(al, al, label=str(alone[al]))

        for orr in or_rel:
            ftm.edge(orr[0], orr[1], label=str(or_rel[orr]), color="red")

        for andr in and_rel:
            ftm.edge(andr[0], andr[1], label=str(and_rel[andr]), color ="blue")

        ftm.render(self.results_dir+'FTM/'+self.system_name)

## Saving results

In [6]:
config_dir = "./config/"

for config_file in sorted(os.listdir(config_dir)):
    system = config_file[:-5]
    tc = ToggleCatcher(system)
    tc.export_results()

------
Exporting boulder results
------
Saving statements to  ./results/statements/boulder.txt
Saving other results to  ./results/kw_file/boulder.json ./results/occurences/boulder.json ./results/count_file/boulder.json


------
Exporting client results
------
Saving statements to  ./results/statements/client.txt
Saving other results to  ./results/kw_file/client.json ./results/occurences/client.json ./results/count_file/client.json


------
Exporting juju results
------
Saving statements to  ./results/statements/juju.txt
Saving other results to  ./results/kw_file/juju.json ./results/occurences/juju.json ./results/count_file/juju.json


------
Exporting kops results
------
Saving statements to  ./results/statements/kops.txt
Saving other results to  ./results/kw_file/kops.json ./results/occurences/kops.json ./results/count_file/kops.json


------
Exporting kubernetes results
------
Saving statements to  ./results/statements/kubernetes.txt
Saving other results to  ./results/kw_file/kuberne

## Table of indicators

In [7]:
print("\\begin{table}")
print("\\begin{tabular}{|c|c|c|c|c|}")
print("\\hline")
print("System & min \\#Files & max \\#Files & min \\#Toggle Point & max \\#Toggle Point \\\\ \\hline")
for config_file in sorted(os.listdir(config_dir)):
    system = config_file[:-5]
    tc = ToggleCatcher(system)
    print(system,
          "& ",  np.min([val for val in tc.count_file.values()]),
          "& ",  np.max([val for val in tc.count_file.values()]),
          "& ",  np.min([val for val in tc.occurences.values()]),
          "& ",  np.max([val for val in tc.occurences.values()]), 
          "\\\\ \\hline")
print("\\end{tabular}")
print("\\end{table}")

\begin{table}
\begin{tabular}{|c|c|c|c|c|}
\hline
System & min \#Files & max \#Files & min \#Toggle Point & max \#Toggle Point \\ \hline
boulder &  1 &  2 &  1 &  4 \\ \hline
client &  0 &  19 &  0 &  94 \\ \hline
juju &  1 &  12 &  1 &  17 \\ \hline
kops &  1 &  11 &  1 &  20 \\ \hline
kubernetes &  0 &  23 &  0 &  106 \\ \hline
loomchain &  0 &  7 &  0 &  28 \\ \hline
\end{tabular}
\end{table}


## Analyse the statements (conditions of if statements)

In [8]:
for config_file in sorted(os.listdir(config_dir)):
    system = config_file[:-5]
    tc = ToggleCatcher(system)
    print("------\n Analysing ", system, "statements\n------")
    print("Alone : ", tc.analyse_statements()[0])
    print("Or relations : ", tc.analyse_statements()[1])
    print("And relations : ", tc.analyse_statements()[2])

------
 Analysing  boulder statements
------
Alone :  {'allowv1registration': 1, 'restrictrsakeysizes': 1, 'fasternewordersratelimit': 3, 'v1disablenewvalidations': 1, 'serverenewalinfo': 3, 'multivafullresults': 1, 'enforcemultiva': 1, 'caaaccounturi': 1, 'caavalidationmethods': 1}
Or relations :  {}
And relations :  {('ecdsaforall', 'expr'): 3, ('expr', 'expr'): 7, ('expr', 'storerevokerinfo'): 2, ('expr', 'mandatorypostasget'): 8, ('caaaccounturi', 'expr'): 1, ('enforcemultiva', 'multivafullresults'): 2}
------
 Analysing  client statements
------
Alone :  {'featureflags': 4, 'expr': 7}
Or relations :  {}
And relations :  {('expr', 'expr'): 1, ('expr', 'featureflags'): 1}
------
 Analysing  juju statements
------
Alone :  {'asynchronouscharmdownloads': 1, 'charmassumes': 2, 'rawk8sspec': 1, 'developermode': 4, 'strictmigration': 1, 'legacyupstart': 2, 'raftbatchfsm': 1, 'raftapileases': 1, 'logerrorstack': 1, 'secrets': 2}
Or relations :  {('branches', 'generations'): 12}
And relati

### Build the FTM with Graphviz

In [13]:
for config_file in sorted(os.listdir(config_dir)):
    system = config_file[:-5]
    tc = ToggleCatcher(system)
    tc.build_ftm()

#### Display an example (or = red, and = blue)

In [15]:
IFrame('./results/FTM/kops.pdf', width=1500, height=500)